# Neural Network from Scratch with NumPy

This notebook builds a complete neural network using only NumPy. No frameworks, no magic—just matrix operations.

**Goal:** Train a network to classify handwritten digits (MNIST).

**Prerequisites:** [perceptron.md](../neural-networks/perceptron.md), [multilayer-networks.md](../neural-networks/multilayer-networks.md)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

np.random.seed(42)

## 1. Load MNIST Data

MNIST contains 70,000 handwritten digits (28×28 pixels each).

In [ ]:
# Load MNIST
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)

# Normalize to [0, 1]
X = X / 255.0

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=10000, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Input shape: {X_train[0].shape}")
print(f"Classes: {np.unique(y_train)}")

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i].reshape(28, 28), cmap='gray')
    ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. Building Blocks

Let's define the core components: activation functions, their derivatives, and loss function.

In [ ]:
# Activation functions

def relu(z):
    """ReLU activation."""
    return np.maximum(0, z)

def relu_derivative(z):
    """Gradient of ReLU."""
    return (z > 0).astype(float)

def softmax(z):
    """Softmax activation (numerically stable)."""
    exp_z = np.exp(z - z.max(axis=1, keepdims=True))
    return exp_z / exp_z.sum(axis=1, keepdims=True)

def cross_entropy_loss(probs, y):
    """
    Cross-entropy loss.
    
    Args:
        probs: (batch_size, num_classes) softmax probabilities
        y: (batch_size,) integer class labels
    """
    batch_size = len(y)
    correct_probs = probs[np.arange(batch_size), y]
    return -np.log(correct_probs + 1e-10).mean()

In [ ]:
# Test softmax
z = np.array([[1, 2, 3], [1, 1, 1]])
probs = softmax(z)
print("Softmax output:")
print(probs)
print(f"Row sums: {probs.sum(axis=1)}")

## 3. The Neural Network Class

Architecture: Input (784) → Hidden (128, ReLU) → Output (10, Softmax)

In [ ]:
class NeuralNetwork:
    """
    Two-layer neural network for classification.
    
    Architecture:
        Input → Linear → ReLU → Linear → Softmax → Output
    """
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        Initialize with He initialization for ReLU.
        """
        # Layer 1: input → hidden
        self.W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2 / input_dim)
        self.b1 = np.zeros(hidden_dim)
        
        # Layer 2: hidden → output
        self.W2 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2 / hidden_dim)
        self.b2 = np.zeros(output_dim)
        
    def forward(self, X):
        """
        Forward pass. Store intermediates for backprop.
        
        Args:
            X: (batch_size, input_dim)
            
        Returns:
            probs: (batch_size, output_dim) softmax probabilities
        """
        # Store input for backprop
        self.X = X
        
        # Layer 1
        self.z1 = X @ self.W1 + self.b1  # (batch, hidden)
        self.a1 = relu(self.z1)           # (batch, hidden)
        
        # Layer 2
        self.z2 = self.a1 @ self.W2 + self.b2  # (batch, output)
        self.probs = softmax(self.z2)          # (batch, output)
        
        return self.probs
    
    def backward(self, y):
        """
        Backward pass. Compute gradients.
        
        Args:
            y: (batch_size,) integer class labels
            
        Returns:
            Dictionary of gradients
        """
        batch_size = len(y)
        
        # Output layer gradient (softmax + cross-entropy combined)
        # dL/dz2 = probs - one_hot(y)
        dz2 = self.probs.copy()
        dz2[np.arange(batch_size), y] -= 1
        dz2 /= batch_size  # Average over batch
        
        # Layer 2 gradients
        dW2 = self.a1.T @ dz2        # (hidden, output)
        db2 = dz2.sum(axis=0)        # (output,)
        
        # Backprop through layer 2
        da1 = dz2 @ self.W2.T        # (batch, hidden)
        
        # Backprop through ReLU
        dz1 = da1 * relu_derivative(self.z1)  # (batch, hidden)
        
        # Layer 1 gradients
        dW1 = self.X.T @ dz1         # (input, hidden)
        db1 = dz1.sum(axis=0)        # (hidden,)
        
        return {'W1': dW1, 'b1': db1, 'W2': dW2, 'b2': db2}
    
    def update(self, grads, learning_rate):
        """
        Update parameters with gradient descent.
        """
        self.W1 -= learning_rate * grads['W1']
        self.b1 -= learning_rate * grads['b1']
        self.W2 -= learning_rate * grads['W2']
        self.b2 -= learning_rate * grads['b2']
    
    def predict(self, X):
        """
        Predict class labels.
        """
        probs = self.forward(X)
        return np.argmax(probs, axis=1)
    
    def accuracy(self, X, y):
        """
        Compute classification accuracy.
        """
        predictions = self.predict(X)
        return (predictions == y).mean()

## 4. Training Loop

Train with mini-batch gradient descent.

In [ ]:
def train(model, X_train, y_train, X_val, y_val, 
          epochs=10, batch_size=32, learning_rate=0.1):
    """
    Train the neural network.
    """
    n_samples = len(X_train)
    n_batches = n_samples // batch_size
    
    history = {'train_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(epochs):
        # Shuffle training data
        indices = np.random.permutation(n_samples)
        X_shuffled = X_train[indices]
        y_shuffled = y_train[indices]
        
        epoch_loss = 0
        
        for i in range(n_batches):
            # Get batch
            start = i * batch_size
            end = start + batch_size
            X_batch = X_shuffled[start:end]
            y_batch = y_shuffled[start:end]
            
            # Forward pass
            probs = model.forward(X_batch)
            loss = cross_entropy_loss(probs, y_batch)
            epoch_loss += loss
            
            # Backward pass
            grads = model.backward(y_batch)
            
            # Update
            model.update(grads, learning_rate)
        
        # Compute metrics
        avg_loss = epoch_loss / n_batches
        train_acc = model.accuracy(X_train[:5000], y_train[:5000])  # Subset for speed
        val_acc = model.accuracy(X_val, y_val)
        
        history['train_loss'].append(avg_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1:2d}: Loss = {avg_loss:.4f}, "
              f"Train Acc = {train_acc:.3f}, Val Acc = {val_acc:.3f}")
    
    return history

In [ ]:
# Create and train the network
model = NeuralNetwork(
    input_dim=784,   # 28x28 pixels
    hidden_dim=128,  # Hidden layer size
    output_dim=10    # 10 digit classes
)

# Split off a validation set
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=5000, random_state=42
)

# Train!
history = train(
    model, X_tr, y_tr, X_val, y_val,
    epochs=15,
    batch_size=64,
    learning_rate=0.1
)

## 5. Visualize Training

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss
ax1.plot(history['train_loss'])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.grid(True)

# Accuracy
ax2.plot(history['train_acc'], label='Train')
ax2.plot(history['val_acc'], label='Validation')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 6. Evaluate on Test Set

In [ ]:
test_acc = model.accuracy(X_test, y_test)
print(f"Test Accuracy: {test_acc:.3f}")
print(f"That's {int(test_acc * len(y_test))} / {len(y_test)} correct!")

In [ ]:
# Visualize predictions on test samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

# Get some random test samples
indices = np.random.choice(len(X_test), 10, replace=False)

for i, (ax, idx) in enumerate(zip(axes.flat, indices)):
    img = X_test[idx].reshape(28, 28)
    pred = model.predict(X_test[idx:idx+1])[0]
    true = y_test[idx]
    
    ax.imshow(img, cmap='gray')
    color = 'green' if pred == true else 'red'
    ax.set_title(f"Pred: {pred}, True: {true}", color=color)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 7. Understanding What the Network Learned

Let's visualize what features the hidden layer learned.

In [ ]:
# Visualize first layer weights
# Each column of W1 is a "feature detector"

fig, axes = plt.subplots(4, 8, figsize=(12, 6))

for i, ax in enumerate(axes.flat):
    if i < model.W1.shape[1]:
        weight = model.W1[:, i].reshape(28, 28)
        ax.imshow(weight, cmap='RdBu_r', vmin=-0.5, vmax=0.5)
    ax.axis('off')

plt.suptitle('First 32 Hidden Unit Weights\n(Red = positive, Blue = negative)', y=1.02)
plt.tight_layout()
plt.show()

## 8. Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Get predictions on test set
predictions = model.predict(X_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Which digits are most confused?
print("\nMost common confusions:")
errors = []
for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 0:
            errors.append((cm[i, j], i, j))

errors.sort(reverse=True)
for count, true, pred in errors[:5]:
    print(f"  True {true} → Predicted {pred}: {count} times")

## 9. Gradient Checking (Optional)

Verify our gradients are correct by comparing with numerical gradients.

In [ ]:
def numerical_gradient(model, X, y, param_name, epsilon=1e-5):
    """
    Compute gradient numerically for verification.
    """
    param = getattr(model, param_name)
    grad = np.zeros_like(param)
    
    # Only check a subset for speed
    indices = np.random.choice(param.size, min(20, param.size), replace=False)
    
    for idx in indices:
        flat_idx = np.unravel_index(idx, param.shape)
        
        # f(x + epsilon)
        param[flat_idx] += epsilon
        probs_plus = model.forward(X)
        loss_plus = cross_entropy_loss(probs_plus, y)
        
        # f(x - epsilon)
        param[flat_idx] -= 2 * epsilon
        probs_minus = model.forward(X)
        loss_minus = cross_entropy_loss(probs_minus, y)
        
        # Restore
        param[flat_idx] += epsilon
        
        # Gradient
        grad[flat_idx] = (loss_plus - loss_minus) / (2 * epsilon)
    
    return grad, indices

# Quick gradient check on a small batch
X_check = X_train[:32]
y_check = y_train[:32]

# Get analytical gradient
model.forward(X_check)
analytical = model.backward(y_check)

# Get numerical gradient for W2
numerical, indices = numerical_gradient(model, X_check, y_check, 'W2')

# Compare
analytical_subset = analytical['W2'].flatten()[indices]
numerical_subset = numerical.flatten()[indices]

diff = np.abs(analytical_subset - numerical_subset).max()
print(f"Max gradient difference (W2): {diff:.2e}")
print("(Should be < 1e-5 for correct implementation)")

## 10. Exercises

Try modifying the network:

1. **Change hidden layer size:** What happens with 32, 64, 256, 512 hidden units?

2. **Add a second hidden layer:** Modify the class to have two hidden layers.

3. **Try different activations:** Replace ReLU with sigmoid or tanh.

4. **Add regularization:** Implement L2 regularization (weight decay).

5. **Implement dropout:** Add dropout between layers.

6. **Learning rate schedule:** Decay the learning rate during training.

In [ ]:
# Exercise 1: Try a larger hidden layer
# model_large = NeuralNetwork(784, 256, 10)
# history = train(model_large, X_tr, y_tr, X_val, y_val, epochs=15, learning_rate=0.1)

## Summary

In this notebook, we built:

1. **Forward pass:** Matrix multiply → activation → matrix multiply → softmax

2. **Backward pass:** Apply chain rule to compute gradients layer by layer

3. **Training loop:** Batch data, forward, backward, update, repeat

Key insights:
- The whole network is just matrix operations
- Backprop reuses intermediate values from the forward pass
- He initialization + ReLU works well in practice
- A simple 2-layer network achieves ~97% on MNIST

**Next:** [02-backprop-from-scratch.ipynb](02-backprop-from-scratch.ipynb) for a deeper dive into the math.